# **Modelo Solución Final**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# **DESCARGAMOS LOS ARCHIVOS DE LA COMPETENCIA**

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

print ("shape of loaded dataframe", df_train.shape)
print ("shape of loaded dataframe", df_test.shape)

shape of loaded dataframe (692500, 12)
shape of loaded dataframe (296786, 12)


#**Funcion clean_data**

1.   Se reemplazan los datos faltantes por la moda
2.   Se normalizan los valores en varias columnas
3.   Se hace una normalización especifica pala la columna ESTU_PRGM_ACADEMICO

In [ ]:
def clean_data(z_copy):
    #Variables Categoricas
  ccols = [i for i in z_copy.columns if not i in z_copy._get_numeric_data()]
  print (ccols)

    #Valor mas repetido
  valor_comun = z_copy[ccols].mode().iloc[0]
  valor_comun

  #Reemplazar datos faltantes con los mas repetidos
  for column in ccols:
    z_copy[column].fillna(valor_comun[column], inplace=True)

  #Modificar las columnas de Si y No por 1 y 0
  z_copy["FAMI_TIENEINTERNET"] = z_copy["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
  z_copy["ESTU_PAGOMATRICULAPROPIO"] = z_copy["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})

#Modificar la columna del Programa academico
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL EN ', '')
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL DE ', '')
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL ', '')
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROGRAMA ', '')

  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INGENIERIA' if x.startswith('INGE') else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LICENCIATURA' if x.startswith('LIC') or x.startswith('Lic') else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CONTADURIA' if x.startswith('CONT')   else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'COMUNICACION' if x.startswith('COMU')   else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DISEÑO' if 'DISE' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DEPORTE' if 'DEPORT'  in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'GASTRONOMIA' if 'GASTRON' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'NUTRICION' if 'NUTRIC' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MATEMATICAS' if ('MATE' in x or 'MATEM' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ESTADISTICA' if ('ESTAD' in x or 'ESTA' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'PERIODISMO' if 'PERIOD' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DERECHO' if 'DERECHO' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LITERATURA' if 'LITER' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MUSICA' if ('MUS' in x or 'MÚS' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'QUIMICA' if ('QUIM' in x or 'QUÍM' in x or 'QU¿M' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ODONTOLOGIA' if 'ODONTOLOG' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ARTES' if 'ARTE' in x else x)

  z_copy = pd.get_dummies(z_copy, columns=ccols, drop_first=True)

  return z_copy

In [ ]:
lentr = len(df_train)

train_labels = df_train[['ID', 'RENDIMIENTO_GLOBAL']]

# Definir las columnas de origen, excluyendo la variable objetivo
source_cols = [i for i in df_train.columns if i != "RENDIMIENTO_GLOBAL"]

# Combinar los datos de entrenamiento y prueba para preprocesamiento
all_data = pd.concat((df_train[source_cols], df_test[source_cols]))
all_data.index = range(len(all_data))

# Limpiar los datos usando la función clean_data
all_data = clean_data(all_data)

#Poner 0 y 1 en ves de False y True
for col in all_data.columns:
    if all_data[col].dtype == 'bool':
        all_data[col] = all_data[col].astype(int)

# Dividir los datos combinados nuevamente en conjuntos de entrenamiento y prueba
train_listos = all_data[all_data['ID'].isin(train_labels['ID'])]
test_listos = all_data[~all_data['ID'].isin(train_labels['ID'])]

train_listos = train_listos.merge(train_labels, on='ID', how='left')


['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO']


C:\Users\57320\AppData\Local\Temp\ipykernel_27004\3430693156.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  z_copy[column].fillna(valor_comun[column], inplace=True)
C:\Users\57320\AppData\Local\Temp\ipykernel_27004\3430693156.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  z_copy["FAMI_TI

In [ ]:
all_data

,ID,PERIODO,ESTU_PRGM_ACADEMICO_3° CICLO NEGOCIOS INTERNACIONALES,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,ESTU_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,ESTU_PRGM_ACADEMICO_ADMINISTRACION & SERVICIO,ESTU_PRGM_ACADEMICO_ADMINISTRACION AERONAUTICA,ESTU_PRGM_ACADEMICO_ADMINISTRACION AGROPECUARIA,...,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,ESTU_PAGOMATRICULAPROPIO_1
0,904256,20212,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,645256,20212,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,308367,20203,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,470353,20195,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,989032,20212,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989281,496981,20195,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
989282,209415,20183,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
989283,239074,20212,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
989284,963852,20195,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
label_encoder = LabelEncoder()
train_listos['RENDIMIENTO_GLOBAL'] = label_encoder.fit_transform(train_listos['RENDIMIENTO_GLOBAL'])

In [ ]:
X = train_listos.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
y = train_listos['RENDIMIENTO_GLOBAL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Entrenamiento del modelo**

In [ ]:
#Modelo Definitivo

rf_model = RandomForestClassifier(
    n_estimators=1750,
    criterion='entropy',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,
    random_state=42
)
rf_model.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', min_samples_leaf=2,
                       n_estimators=1750, random_state=42)

In [ ]:
# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Random Forest Reporte de Clasificacion:\n", classification_report(y_test, y_pred))


Random Forest Accuracy: 0.42031768953068593
Random Forest Reporte de Clasificacion:
               precision    recall  f1-score   support

           0       0.53      0.63      0.57     35165
           1       0.43      0.57      0.49     34573
           2       0.32      0.23      0.27     34259
           3       0.32      0.24      0.28     34503

    accuracy                           0.42    138500
   macro avg       0.40      0.42      0.40    138500
weighted avg       0.40      0.42      0.40    138500



# **Creación de archivo con las variables necesarias**

In [ ]:
ids = test_listos['ID']
new_data_processed = test_listos.drop('ID', axis=1)


predictions = rf_model.predict(new_data_processed)
test_listos['RENDIMIENTO_GLOBAL'] = predictions
test = test_listos[['ID', 'RENDIMIENTO_GLOBAL']]
mapping = {2: 'medio-alto', 0: 'alto', 1: 'bajo', 3: 'medio-bajo'}
# Aplicar el mapeo a la columna RENDIMIENTO_GLOBAL
test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)

# Guardar el DataFrame de envío en un archivo CSV
test.to_csv('archivo_final3.csv', index=False)

C:\Users\57320\AppData\Local\Temp\ipykernel_27004\3727749520.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_listos['RENDIMIENTO_GLOBAL'] = predictions
C:\Users\57320\AppData\Local\Temp\ipykernel_27004\3727749520.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_listos['RENDIMIENTO_GLOBAL'] = predictions
C:\Users\57320\AppData\Local\Temp\ipykernel_27004\3727749520.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =